# Cryptanalysis | Affine Cipher

In [31]:
from scipy.optimize import minimize
import numpy as np

In [47]:
actual_freq = {
    'e': 0.127, 't': 0.0906, 'a': 0.0817, 'o': 0.0751, 'i': 0.0697,
    'n': 0.0675, 's': 0.0633, 'h': 0.0609, 'r': 0.0599, 'd': 0.0425,
    'l': 0.0403, 'c': 0.0278, 'u': 0.0276, 'm': 0.0241, 'w': 0.0236,
    'f': 0.0223, 'g': 0.0202, 'y': 0.0197, 'p': 0.0193, 'b': 0.0129,
    'v': 0.0098, 'k': 0.0077, 'j': 0.0015, 'x': 0.0015, 'q': 0.0010,
    'z': 0.0007
}
res_a = 0
res_b = 0
res_mse = float('inf')
decrypted = ""

# computng euclid's gcd
def egcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = egcd(b % a, a)
        return g, y - (b // a) * x, x

# modulo inverse
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % m

# mean squarred error
def find_mse(plaintext):
  decrypt_frequency = {}
  mse = 0
  for char in plaintext:
      decrypt_frequency[char] = decrypt_frequency.get(char, 0) + 1
  for char in decrypt_frequency:
    decrypt_frequency[char]/=len(plaintext)
    mse += ((decrypt_frequency[char]-actual_freq[char])**2)
  return mse

# decrypt with key using function dK (y) = a^−1 (y − b) mod 26
def decrypt_affine(ciphertext, a, b):
    plaintext = ''

    for char in ciphertext:
        if char.isalpha():
            char_num = ord(char) - ord('a')
            decrypted_char_num = (modinv(a, 26) * (char_num - b)) % 26
            plaintext += chr(decrypted_char_num + ord('a'))
        else:
            plaintext += char
    return plaintext

# computing frequencies for subsititution, most frequent to 'e' and then 't' and so on
def analyze_ciphertext_frequency(ciphertext):
    frequency = {}
    for char in ciphertext:
        if char.isalpha():
            char = char.lower()
            frequency[char] = frequency.get(char, 0) + 1
    return frequency

# solving ax + b = k for 'e' and 't' substitutions
def solve_eq(x1,x2):
  a = ((x2-x1)) % 26
  a = a * modinv(15, 26)
  a = a %26
  b = ( x1 - 4*a) % 26
  return a,b

# key computing based on mse
def find_possible_key(frequency, most_frequent_letter, target_letter, ciphertext):
    global res_a
    global res_b
    global res_mse
    global decrypted
    x1_candidate = (ord(target_letter) - ord('a')) % 26
    for letter in frequency:
      if(ord(letter) - ord('a') != x1_candidate):
        x2_candidate = (ord(letter) - ord('a')) % 26
        # print(f"{x1_candidate},{x2_candidate}")
        a_candidate, b_candidate = solve_eq(x1_candidate,x2_candidate)
        if egcd(a_candidate, 26)[0] == 1:
            pt = decrypt_affine(ciphertext, a_candidate, b_candidate)
            mse = find_mse(pt)
            # print(mse)
            # print(res_mse)
            if mse < res_mse:
              res_mse = mse
              res_a = a_candidate
              res_b = b_candidate
              decrypted = pt


ciphertext = input("Enter ciphertext: ")
# "FMXVEDKAPHFERBNDKRXRSREFMORUDSDKDVSHVUFEDKAPRKDLYEVLRHHRH"
ciphertext = ciphertext.lower()
print("Ciphertext: ",ciphertext)
frequency = analyze_ciphertext_frequency(ciphertext)
most_frequent_letters = sorted(frequency, key=frequency.get, reverse=True)

target_letter_e = 'e'
target_letter_t = 't'

for letter_t in most_frequent_letters:
  find_possible_key(most_frequent_letters, target_letter_e, letter_t, ciphertext)


print(f"Potential key found:\n a = {res_a}, \n b = {res_b} \n with mse = {res_mse}")
print("Decrypted Text:")
print(decrypted)



Enter ciphertext: lvhhmlznzaluoklvyzingklhojnndofoaxofklygfyoinrffyilhnygngpohrphhpfypyuklrnhfcnaoinmlzpupzmnanbapynfynhvuunanglmafmvkhpfyulvkooilohzvmmnanglyxmafhmnzofulkvylaofvzigfjylygpojlhzfrrlygngofgnhoafxpohnkupyhonlgoinmapclonfmnalofampoohsvabislhnglhoafsfopzgpanzongoinzaluopyofnlaoihlorfhminanofsvayvmloalzdpybholopfypyzlysnaallvhoalkplzfyuparngkfhhfuhpbylkjpoimnanbapynlobro
Ciphertext:  lvhhmlznzaluoklvyzingklhojnndofoaxofklygfyoinrffyilhnygngpohrphhpfypyuklrnhfcnaoinmlzpupzmnanbapynfynhvuunanglmafmvkhpfyulvkooilohzvmmnanglyxmafhmnzofulkvylaofvzigfjylygpojlhzfrrlygngofgnhoafxpohnkupyhonlgoinmapclonfmnalofampoohsvabislhnglhoafsfopzgpanzongoinzaluopyofnlaoihlorfhminanofsvayvmloalzdpybholopfypyzlysnaallvhoalkplzfyuparngkfhhfuhpbylkjpoimnanbapynlobro
Potential key found:
 a = 7, 
 b = 11 
 with mse = 0.0033890524539400995
Decrypted Text:
ausspacecraftlaunchedlastweektotrytolandonthemoonhasendeditsmissioninflamesoverthepacificperegrineonesufferedapropulsionfaultthatscupperedanyprosp

**a = 7, b=11**